# Informe final

Datos de los integrantes, tecnologias usadas, fecha, materia, 
Que estructura tiene nuestro proyecto
Como se ejecuta. Pasos para 
Que datos guardamos en cada base, como lo guardamos y porque ? (ventajas, 
Consultas combinadas ejemplo 